# 좌표 X, Y 결측치 처리
- '시군구', '번지'를 합쳐 하나의 feature '시군구 번지' 생성
- 크롤링을 통해 좌표를 찾는다.
- '시군구 번지'에 맞게 결측치를 처리한다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import time
import pickle
import gc

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

# 맥에서 글씨 깨짐 방지
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## DataLoad

In [3]:
df = pd.read_csv('../data/train_test_data.csv')
df

/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_48675/2817759318.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/train_test_data.csv')


,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000.0,0
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500.0,0
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,91500.0,0
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,130000.0,0
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,117000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202307,19,13,2014,...,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.10672,37.618870,2014-09-01 13:05:03.0,NaN,1
1128090,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.62,202307,25,12,2014,...,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.10672,37.618870,2014-09-01 13:05:03.0,NaN,1
1128091,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.65,202308,27,12,2014,...,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.10672,37.618870,2014-09-01 13:05:03.0,NaN,1
1128092,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.94,202309,2,18,2014,...,1568.0,의무,2015-09-09 15:30:27.0,Y,N,127.10672,37.618870,2014-09-01 13:05:03.0,NaN,1


In [4]:
df.shape

(1128094, 53)

## X, Y 좌표가 없는 데이터만 추출

In [5]:
tmp = df[['시군구', '번지', '아파트명', '좌표X', '좌표Y']]
x_y_nan = tmp[tmp['좌표X'].isna()]

In [6]:
x_y_nan

,시군구,번지,아파트명,좌표X,좌표Y
975,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN
976,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN
977,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN
978,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN
979,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN
...,...,...,...,...,...
1128075,서울특별시 중랑구 중화동,450,한신,NaN,NaN
1128076,서울특별시 중랑구 중화동,450,한신,NaN,NaN
1128077,서울특별시 중랑구 중화동,450,한신,NaN,NaN
1128078,서울특별시 중랑구 중화동,450,한신,NaN,NaN


In [7]:
x_y_nan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 876232 entries, 975 to 1128079
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   시군구     876232 non-null  object 
 1   번지      876157 non-null  object 
 2   아파트명    874096 non-null  object 
 3   좌표X     0 non-null       float64
 4   좌표Y     0 non-null       float64
dtypes: float64(2), object(3)
memory usage: 40.1+ MB


In [8]:
a = x_y_nan['시군구']
b = x_y_nan['번지']
x_y_nan['시군구 번지'] = a+' '+b

/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_48675/1203648578.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_y_nan['시군구 번지'] = a+' '+b


In [9]:
x_y_nan

,시군구,번지,아파트명,좌표X,좌표Y,시군구 번지
975,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN,서울특별시 강남구 개포동 189
976,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN,서울특별시 강남구 개포동 189
977,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN,서울특별시 강남구 개포동 189
978,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN,서울특별시 강남구 개포동 189
979,서울특별시 강남구 개포동,189,개포주공4단지,NaN,NaN,서울특별시 강남구 개포동 189
...,...,...,...,...,...,...
1128075,서울특별시 중랑구 중화동,450,한신,NaN,NaN,서울특별시 중랑구 중화동 450
1128076,서울특별시 중랑구 중화동,450,한신,NaN,NaN,서울특별시 중랑구 중화동 450
1128077,서울특별시 중랑구 중화동,450,한신,NaN,NaN,서울특별시 중랑구 중화동 450
1128078,서울특별시 중랑구 중화동,450,한신,NaN,NaN,서울특별시 중랑구 중화동 450


In [10]:
print(x_y_nan.shape)
print(len(x_y_nan['시군구 번지'].unique()))
locates = x_y_nan['시군구 번지'].unique()

(876232, 6)
8204


## 크롤링

In [ ]:
url = 'https://www.findlatlng.org/'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(20)

# '힐스테이트 서초 젠트리스': (127.05347005393612, 37.45628519999)
re = dict()
cnt = 1

tbar = tqdm(locates)
for locate in tbar:
    driver.find_element(By.CLASS_NAME, 'form-control').clear()
    driver.find_element(By.CLASS_NAME, 'form-control').click()
    driver.find_element(By.CLASS_NAME, 'form-control').send_keys(locate)
    driver.find_element(By.CLASS_NAME, 'btn').click()
    time.sleep(1)
    container = driver.find_elements(By.CLASS_NAME, 'container-fluid')
    try:
        data = container[1].text.split('\n')[1].split(' / ')
        X = data[1][16:]
        Y = data[0][15:]
        re[locate] = (float(X), float(Y))
    except:
        continue

    if cnt % 500 == 0:
        finded_x_y_df = pd.DataFrame(re).transpose()
        finded_x_y_df.columns = ['좌표X', '좌표Y']
        finded_x_y_df.to_csv(f'./data/finded_x_y_{cnt//500}.csv')

        del re
        del finded_x_y_df
        gc.collect()
        re = dict()
    
    cnt += 1

finded_x_y_df = pd.DataFrame(re).transpose()
finded_x_y_df.columns = ['좌표X', '좌표Y']
finded_x_y_df.to_csv(f'./data/finded_x_y_last.csv')

In [12]:
# 찾은 X, Y 좌표 데이터
X_Y_df = pd.concat([pd.read_csv(f'../data/x_y/finded_x_y_{i}.csv') for i in range(1, 18)], axis=0)

X_Y_df.columns = ['시군구 번지', '좌표X', '좌표Y']
X_Y_df.reset_index(drop=True)

,시군구 번지,좌표X,좌표Y
0,서울특별시 강남구 개포동 189,127.073169,37.486331
1,서울특별시 강남구 개포동 185,127.071668,37.488876
2,서울특별시 강남구 개포동 655-1,127.052680,37.481693
3,서울특별시 강남구 개포동 649,127.054174,37.484515
4,서울특별시 강남구 개포동 1164-30,127.054793,37.474820
...,...,...,...
8199,서울특별시 구로구 고척동 97,126.856619,37.499206
8200,서울특별시 동작구 흑석동 253-89,126.960784,37.500314
8201,서울특별시 마포구 신수동 400-1,126.935946,37.544875
8202,서울특별시 성동구 송정동 67-5,127.063427,37.548716


## 결측치 채워넣기

In [13]:
# 시군구, 번지가 합쳐진 하나의 변수 생성

a = df['시군구']
b = df['번지']
df['시군구 번지'] = a+' '+b

df

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,시군구 번지
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000.0,0,서울특별시 강남구 개포동 658-1
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,123500.0,0,서울특별시 강남구 개포동 658-1
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,91500.0,0,서울특별시 강남구 개포동 658-1
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,130000.0,0,서울특별시 강남구 개포동 658-1
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,117000.0,0,서울특별시 강남구 개포동 658-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202307,19,13,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.10672,37.618870,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816
1128090,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.62,202307,25,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.10672,37.618870,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816
1128091,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.65,202308,27,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.10672,37.618870,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816
1128092,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.94,202309,2,18,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.10672,37.618870,2014-09-01 13:05:03.0,NaN,1,서울특별시 중랑구 신내동 816


In [14]:
# '시군구 번지'에 맞춰 X, Y 를 매핑

X_mapping = pd.Series({locate: X for locate, X in zip(X_Y_df['시군구 번지'], X_Y_df['좌표X'])})
Y_mapping = pd.Series({locate: Y for locate, Y in zip(X_Y_df['시군구 번지'], X_Y_df['좌표Y'])})

In [15]:
X_mapping

서울특별시 강남구 개포동 189        127.073169
서울특별시 강남구 개포동 185        127.071668
서울특별시 강남구 개포동 655-1      127.052680
서울특별시 강남구 개포동 649        127.054174
서울특별시 강남구 개포동 1164-30    127.054793
                            ...    
서울특별시 구로구 고척동 97         126.856619
서울특별시 동작구 흑석동 253-89     126.960784
서울특별시 마포구 신수동 400-1      126.935946
서울특별시 성동구 송정동 67-5       127.063427
서울특별시 중구 입정동 277         126.993077
Length: 8203, dtype: float64

In [16]:
# 원본 데이터에 적용
df['좌표X'] = df['좌표X'].fillna(df['시군구 번지'].map(X_mapping))
df['좌표Y'] = df['좌표Y'].fillna(df['시군구 번지'].map(Y_mapping))

In [17]:
# 힐스테이트 서초 젠트리스는 번지가 없기에 따로 작업

df[df['좌표X'].isna()]['아파트명'].unique()

array(['힐스테이트 서초 젠트리스'], dtype=object)

In [18]:
# 직접 찾은 좌표 적용

hill_X_mapping = pd.Series({'힐스테이트 서초 젠트리스': 127.05340799855254})
hill_Y_mapping = pd.Series({'힐스테이트 서초 젠트리스': 37.457157542923255})

df['좌표X'] = df['좌표X'].fillna(df['아파트명'].map(hill_X_mapping))
df['좌표Y'] = df['좌표Y'].fillna(df['아파트명'].map(hill_Y_mapping))

In [19]:
print(df['좌표X'].isna().sum())
print(df['좌표Y'].isna().sum())

0
0


In [ ]:
df.to_csv('../data/preprocessed/X_Y_preprocessed.csv', index=False)